## This will be our initial attempt to develop a script to scrape the website of choice.

In [15]:
import requests 
from bs4 import BeautifulSoup 
    
def getdata(url): 
    r = requests.get(url) 
    return r.text 
        
htmldata = getdata("https://en.wikipedia.org/wiki/Cheese") 
soup = BeautifulSoup(htmldata, 'html.parser')

tgt_img_urls = []
    
for item in soup.find_all('img'):
    print(item['src'])
    https_concat = 'https:'+(item['src'])
    tgt_img_urls.append(https_concat)
    

/static/images/icons/wikipedia.png
/static/images/mobile/copyright/wikipedia-wordmark-en.svg
/static/images/mobile/copyright/wikipedia-tagline-en.svg
//upload.wikimedia.org/wikipedia/en/thumb/1/1b/Semi-protection-shackle.svg/20px-Semi-protection-shackle.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/4/47/Sound-icon.svg/20px-Sound-icon.svg.png
//upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Cheese_platter.jpg/260px-Cheese_platter.jpg
//upload.wikimedia.org/wikipedia/commons/thumb/b/b4/Clara_Peeters_-_Still_Life_with_Cheeses%2C_Almonds_and_Pretzels.jpg/260px-Clara_Peeters_-_Still_Life_with_Cheeses%2C_Almonds_and_Pretzels.jpg
//upload.wikimedia.org/wikipedia/commons/thumb/f/f2/Hartkaese_HardCheeses.jpg/220px-Hartkaese_HardCheeses.jpg
//upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Ricotta_affumicata_della_sila.jpg/220px-Ricotta_affumicata_della_sila.jpg
//upload.wikimedia.org/wikipedia/commons/thumb/4/48/Formaggi.JPG/220px-Formaggi.JPG
//upload.wikimedia.org/wikipedia/comm

In [12]:
# credit to Nate who found this and made it work for us!
import os
import requests
import shutil

# create a directory to store what we will download
# user will need to define their own desired destination folder
os.mkdir('C:\\Users\\hanse\\Documents\\GitHub\\Algorithmic-Bias\\Web-Scraping\\image-folder')
os.chdir('C:\\Users\\hanse\\Documents\\Github\\Algorithmic-Bias\\Web-Scraping\\image-folder')

# more robust way of doing stuff above
# def getdata(url):
#     folder_name = input('File Name:- ')
#     if not os.path.exists(folder_name):
#         os.makedirs(folder_name) 
#     os.chdir(f'{folder_name}')
#     r = requests.get(url) 
#     return r.text

urls = ['https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Cheese_platter.jpg/520px-Cheese_platter.jpg'
       ,'https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Ricotta_affumicata_della_sila.jpg/440px-Ricotta_affumicata_della_sila.jpg'
       ,'https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Formaggi.JPG/440px-Formaggi.JPG']
for x in tgt_img_urls:
    filename = x.split("/")[-1]

# Open the url image, set stream to True, this will return the stream content.
    r = requests.get(x, stream = True)

# Check if the image was retrieved successfully
    if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True
    
    # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)

        print('Image sucessfully Downloaded: ',filename)
    else:
        print('Image Couldn\'t be retreived')

SSLError: HTTPSConnectionPool(host='static', port=443): Max retries exceeded with url: /images/icons/wikipedia.png (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1129)')))